In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
import evaluate
import json

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import string 
import json

def my_split(x):
  temp = x.split()
  result = []
  for tok in temp:
    accum = ''
    for t in tok: 
      if t not in string.punctuation:
        accum += t
      else:
        if len(accum) > 0:
          result.append(accum)
        accum = ''
        result.append(t)
    if len(accum) > 0:
      result.append(accum)
  return result

# Env

In [ ]:
%cd /content

%env PYTHONPATH=

!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!bash ./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/

In [ ]:
!conda install --channel defaults conda python=3.8.10 --yes
!conda update --channel defaults --all --yes

In [ ]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages") #?

In [ ]:
!yes | bash /content/drive/MyDrive/PLBART-main/install_env.sh

# Generate comments and eval BLEU-4

In [ ]:
%cd /content/drive/MyDrive/PLBART-main/scripts/code_to_text/
!bash generate.sh 0 python base checkpoint{}.pt test


In [ ]:
%cd /content/drive/MyDrive/PLBART-main/scripts/code_to_text/
!bash generate.sh 0 python base checkpoint{}.pt valid

In [ ]:
%cd /content/drive/MyDrive/PLBART-main/scripts/code_to_text/
!bash generate.sh 0 python finetuned checkpoint{}.pt test


In [ ]:
%cd /content/drive/MyDrive/PLBART-main/scripts/code_to_text/
!bash generate.sh 0 python finetuned checkpoint{}.pt valid

# Other metrics

In [ ]:
references = []
with open('/content/drive/MyDrive/mCoNaLa/mconala_final', 'r', encoding='utf-8') as f:
  for line in f:
    t = " ".join(my_split(line.strip().replace('`', '').lower()))
    references.append(t)

assert len(references) == 345

In [ ]:
results = '/content/drive/MyDrive/PLBART-main/checkpoints/finetuned_python_ru_XX_10/output_checkpoint5.pt_test.hyp'

predictions = []
with open(results, 'r', encoding='utf-8') as f:
  for line in f:
     pred = " ".join(my_split(line.strip().lower()))
     predictions.append(pred)

assert len(predictions) == 345

In [ ]:
bleu = evaluate.load("bleu", max_order=4, smooth=True)
results = bleu.compute(predictions=predictions, references=references)
print(results)

rouge_l = evaluate.load('rouge')
results = rouge_l.compute(predictions=predictions,
                        references=references,
                        tokenizer=lambda x: x.split(), 
                        rouge_types=['rougeL'])
print(results)

chrf = evaluate.load("chrf")
results = chrf.compute(predictions=predictions,
                      references=references,
                      lowercase=True)
print(results)

results = chrf.compute(predictions=predictions,
                      references=references,
                      char_order=1,
                      lowercase=True)
print(results)

{'bleu': 0.01213241159128543, 'precisions': [0.14181486548107616, 0.038419913419913417, 0.015302727877578177, 0.0017226528854435831], 'brevity_penalty': 0.6232121945051228, 'length_ratio': 0.6789473684210526, 'translation_length': 2193, 'reference_length': 3230}
{'rougeL': 0.10499445693089801}
{'score': 14.989183216016237, 'char_order': 6, 'word_order': 0, 'beta': 2}
{'score': 45.718353044240025, 'char_order': 1, 'word_order': 0, 'beta': 2}


In [ ]:
references_v = []
with open('/content/drive/MyDrive/PLBART-main/data/codeXglue/code-to-text/python/valid.jsonl', 'r', encoding='utf-8') as f:
  for line in f:
    t = " ".join(json.loads(line)['docstring_tokens'])
    references_v.append(t)

In [ ]:
results = '/content/drive/MyDrive/PLBART-main/checkpoints/finetuned_python_ru_XX_10/output_checkpoint9.pt_valid.hyp'

predictions_v = []
with open(results, 'r', encoding='utf-8') as f:
  for line in f:
     pred = " ".join(my_split(line.strip().lower()))
     predictions_v.append(pred)


bleu = evaluate.load("bleu", max_order=4, smooth=True)
results = bleu.compute(predictions=predictions_v, references=references_v)
print(results)

rouge_l = evaluate.load('rouge')
results = rouge_l.compute(predictions=predictions_v,
                        references=references_v,
                        tokenizer=lambda x: x.split(), 
                        rouge_types=['rougeL'])
print(results)

chrf = evaluate.load("chrf")
results = chrf.compute(predictions=predictions_v,
                      references=references_v,
                      lowercase=True)
print(results)

results = chrf.compute(predictions=predictions_v,
                      references=references_v,
                      char_order=1,
                      lowercase=True)
print(results)

{'bleu': 0.011926906657339737, 'precisions': [0.20289046653144016, 0.025618459991102686, 0.006713078070611636, 0.002023604188453233], 'brevity_penalty': 0.7316650975310205, 'length_ratio': 0.7619440133430359, 'translation_length': 118320, 'reference_length': 155287}
{'rougeL': 0.17902947042388812}
{'score': 18.571743568835018, 'char_order': 6, 'word_order': 0, 'beta': 2}
{'score': 50.528588912353115, 'char_order': 1, 'word_order': 0, 'beta': 2}
